In [1]:
#custom libraries
from data_import import *
from data_preprocess import *
from data_preprocess_BERT import *

In [2]:
#set filepath constants
NLP_TOPIC_RECOGNITION_FILEPATH = '../NLP Topic Recognition'
NLP_MODEL_FILEPATH = '../NLP Model Training/model_100K_glove'
TEXTUAL_RAW_DATA_FILEPATH = '../Data/textual_raw'

OUTPUT_DATASET_FILEPATH = "Dataset_sentiment_100K_glove_filtered_full_user_weights_user_averaged.csv"
# OUTPUT_DATASET_FILEPATH = "Dataset_sentiment_100K_glove_filtered_full_user_weights.csv"
# OUTPUT_DATASET_FILEPATH = "Dataset_sentiment_100K_glove_filtered_2g_user_weights.csv"
# OUTPUT_DATASET_FILEPATH_PREV = "Dataset_sentiment_BERT_trainable.csv"
# OUTPUT_DATASET_FILEPATH = "Dataset_sentiment_BERT_trainable_full.csv"

In [3]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.append(NLP_TOPIC_RECOGNITION_FILEPATH)
from topic_recognition import *
#from NLP_class import *

*** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.
text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


In [4]:
from tensorflow import keras
model = keras.models.load_model(NLP_MODEL_FILEPATH)

In [6]:
# returns a dictionary of the form {'username': (sentiment_score_sum, occurances)}
def get_users(paths):
    df = pd.DataFrame()
    for path in paths:
        text_data = preprocess_textual(path)
        df = pd.concat([df, text_data])
    
    #take the set of all usernames
    ini_set = set(df['username'].values)
    #initialized as neutral (0.5) and 1 occurance
    users = dict.fromkeys(ini_set, (0.5,1))
    return users

#updates username's sentiment score values after recieving username's tweet
def user_update(username, sentiment):
    #update 
    (s,t) = users[username]
    (s_new, t_new) = (s + sentiment, t + 1)
    users[username] = (s_new, t_new)
    
    #sentiment calculation sentiment_weighted has range(-1, 1)
    #the extreme values are seen when a positive (on average) user tweets something really negative and the converse
    sentiment_weighted = sentiment - s/t
    return sentiment_weighted

In [ ]:
#Create a bert_preprocess object that automatically sets tokenizers and pretrained bert model classifier
bert = BERT_preprocess()

In [ ]:
# lol = text.df[:100]
# lol['sentiment']=bert.predict(list(lol['text'].values))
# pd.options.display.max_colwidth = 400
# print(lol[['sentiment','text']])
# lol = text.df
# list(text.df['text'].values)
# start = timer()
# lol.text.map(lambda x: bert.predict([x]))
# print(timer() - start)

# Sentiment analysis on Tweets

## BERT

In [ ]:
#fetch btc price data
btc_data = preprocess_BTC()

#itterate folder for all raw textual data
txt_folder = Path(TEXTUAL_RAW_DATA_FILEPATH).rglob('*.csv')
paths = [x for x in txt_folder]

#initialize username dictionary for user wieghted sentiment scores
#users = get_users(paths)

In [ ]:
txt_folder = Path(TEXTUAL_RAW_DATA_FILEPATH).rglob('*.csv')
paths = [x for x in txt_folder]
print(paths)

In [ ]:
i = 1

# final dataframe (initialize or load)
# df = pd.DataFrame()
df = pd.read_csv(OUTPUT_DATASET_FILEPATH, index_col = 0)
print(df)
start = timer()
for path in paths:
    if i<47:
        i+=1
        continue
    message = "Iterration " + str(i) + " of " + str(len(paths))
    print(message)
    text_data = preprocess_textual(path)
    
    #initialize an NLP_preprocess object for the current dataset
    text = NLP_preprocess(text_data)
    
    #print(text.df.shape[0])
    #perform some spam_filtering on the current dataset
    text.spam_filtering()
    #print(text.df.shape[0])
    
    #calculate the sentiment score for every tweet in the current dataset
    text.df['sentiment'] = bert.predict(list(text.df['text'].values))
    
    #add an "hour" column that contains the datetime in hourly resolution
    text.df["hour"] =  text.df.date.map(lambda x: x[:-6])
    #print(text.df)
    
    #get the mean and median sentiment per hour
    timetable = pd.DataFrame(text.df.groupby(['hour']).sentiment.apply(np.mean).values, columns = ["Bitcoin"])
    timetable2 = pd.DataFrame(text.df.groupby(['hour']).sentiment.apply(np.median).values, columns = ["Bitcoin_median"])
    
    #sync the price and sentiment dataset so that on each row price follows the sentiment
    timetable = timetable[:-1]
    timetable2 = timetable2[:-1]
    btc_connected = connect_datasets(text.df, btc_data)[1:]
    
    i += 1
    #if mismatching sizes disregard current dataset
    if len(timetable.index) != len(btc_connected.index):
        continue
    #otherwise combine them into a single dataset 
    timetable.index = btc_connected.index
    timetable2.index = btc_connected.index
    timetable = pd.concat([timetable['Bitcoin'], timetable2['Bitcoin_median'], btc_connected['Open']], axis = 1)
    #print(timetable)
    
    df = pd.concat([df, timetable], axis = 0)   
    print(df)
    
    #save final dataset
    df.to_csv(OUTPUT_DATASET_FILEPATH)
print(timer() - start)


In [ ]:
df.to_csv(OUTPUT_DATASET_FILEPATH)

In [ ]:
df

## Plain Glove

In [7]:
#fetch btc price data
btc_data = preprocess_BTC()

#itterate folder for all raw textual data
txt_folder = Path(TEXTUAL_RAW_DATA_FILEPATH).rglob('*.csv')
paths = [x for x in txt_folder]

#initialize username dictionary for user wieghted sentiment scores
users = get_users(paths)

In [9]:
#itterate folder for all raw textual data
txt_folder = Path(TEXTUAL_RAW_DATA_FILEPATH).rglob('*.csv')
paths = [x for x in txt_folder]

i = 1

# final dataframe
df = pd.DataFrame()


start = timer()
for path in paths:

    message = "Iterration " + str(i) + " of " + str(len(paths))
    print(message)
    text_data = preprocess_textual(path)
    
#     old approach
#     timetable = topic_recognition(text_data, flagged_usernames = [], display = message)
#     print(timetable)
    
    #initialize an NLP_preprocess object for the current dataset
    text = NLP_preprocess(text_data)
    
    print(text.df.shape[0])
    text.flag_users()
    
    
    #basic nlp preprocessing (rm stopwords, rm punctuation etc.)
    text.preprocess_data()
    #perform some spam filtering
    text.spam_filtering()
    
    print(text.df.shape[0])
    
    #import default tokenizer for the model
    text.import_tokenizer()
    #tokenize and pad text sequences of the dataset
    df_padded = text.tokenize_and_pad(train = False)
    
    #apply the model on the padded sequences to get sentiment
    text.df["sentiment"] = (model.predict(df_padded))
    #print(text.df.index)
    

    #try: weight messeges by the average optimism/pessimism of each user
    #this might have an impact because there are about 5.5 messages per user,
    #so there is some history about them
    text.df['weighted'] = text.df.index.map(lambda x: user_update(text.df.loc[x]['username'],  text.df.loc[x]['sentiment'])) 
    #print(text.df)
    #add an "hour" column that contains the datetime in hourly resolution
    text.df["hour"] =  text.df.date.map(lambda x: x[:-6])
    
    #old
#     timetable = pd.DataFrame(text.df.groupby(['hour']).sentiment.apply(np.mean).values, columns = ["Bitcoin"])
#     timetable2 = pd.DataFrame(text.df.groupby(['hour']).sentiment.apply(np.median).values, columns = ["Bitcoin_median"])
    
    #we can eliminate the big effect of users that sent multiple messages by first averaging per user per hour
    user_averaged = text.df.groupby(['hour', 'username'], as_index=False).weighted.apply(np.mean)
    #and then get the mean and median weighted sentiment per hour
    timetable = pd.DataFrame(user_averaged.groupby(['hour']).weighted.apply(np.mean).values, columns = ["Bitcoin"])
    timetable2 = pd.DataFrame(user_averaged.groupby(['hour']).weighted.apply(np.median).values, columns = ["Bitcoin_median"])
    
    #get the mean and median weighted sentiment per hour
#     timetable = pd.DataFrame(text.df.groupby(['hour']).weighted.apply(np.mean).values, columns = ["Bitcoin"])
#     timetable2 = pd.DataFrame(text.df.groupby(['hour']).weighted.apply(np.median).values, columns = ["Bitcoin_median"])
     
    #sync the price and sentiment dataset so that on each row price follows the sentiment
    timetable = timetable[:-1]
    timetable2 = timetable2[:-1]
    btc_connected = connect_datasets(text.df, btc_data)[1:]
    
    i += 1
    #if mismatching sizes disregard current dataset
    if len(timetable.index) != len(btc_connected.index):
        continue
    #otherwise combine them into a single dataset 
    timetable.index = btc_connected.index
    timetable2.index = btc_connected.index
    timetable = pd.concat([timetable['Bitcoin'], timetable2['Bitcoin_median'], btc_connected['Open']], axis = 1)
    print(timetable)
    
    df = pd.concat([df, timetable], axis = 0)   
    #print(df)

#save final dataset
df.to_csv(OUTPUT_DATASET_FILEPATH)
print(timer() - start)


Iterration 1 of 65
38159
24276
                Bitcoin  Bitcoin_median         Open
2017-09-13 01  0.005622        0.012747  4128.005439
2017-09-13 02  0.016173        0.017391  4066.801167
2017-09-13 03 -0.011676        0.000703  3999.225167
2017-09-13 04  0.004191        0.004836  3959.409667
2017-09-13 05  0.064276        0.058035  3995.046667
2017-09-13 06  0.027967        0.026831  4005.132264
2017-09-13 07  0.021788        0.017369  3982.990000
2017-09-13 08  0.022074        0.025724  3951.898500
2017-09-13 09 -0.001307        0.016698  3969.537119
2017-09-13 10  0.009001        0.015788  3926.753000
2017-09-13 11 -0.038504        0.000344  3839.378167
2017-09-13 12 -0.014517        0.007375  3833.916333
2017-09-13 13 -0.013050        0.000444  3868.011000
2017-09-13 14  0.012666        0.021143  3794.985667
2017-09-13 15  0.019341        0.030837  3820.265667
2017-09-13 16  0.016617        0.022011  3854.942500
2017-09-13 17 -0.007403       -0.000691  3932.136833
2017-09-13 18  

30501
19269
                Bitcoin  Bitcoin_median         Open
2017-10-21 01  0.014038        0.033624  6020.570167
2017-10-21 02 -0.018180        0.000848  6071.602833
2017-10-21 03  0.032518        0.053828  6085.635965
2017-10-21 04  0.042127        0.055414  6096.996667
2017-10-21 05  0.043895        0.060254  6059.425424
2017-10-21 06  0.030260        0.037595  6074.816167
2017-10-21 07  0.062016        0.065809  6093.565965
2017-10-21 08  0.070110        0.061488  6073.152167
2017-10-21 09  0.066692        0.060798  6075.103667
2017-10-21 10  0.070559        0.069154  6094.042500
2017-10-21 11  0.072614        0.104381  6102.605500
2017-10-21 12  0.059360        0.079055  6111.947000
2017-10-21 13  0.060133        0.045908  6104.586167
2017-10-21 14  0.049741        0.061139  6139.857000
2017-10-21 15  0.074845        0.075855  6146.333000
2017-10-21 16  0.068419        0.083291  6146.445167
2017-10-21 17  0.049002        0.058972  5995.315333
2017-10-21 18  0.047306        0.0

70689
48579
                Bitcoin  Bitcoin_median          Open
2017-12-14 01  0.017821        0.039902  16271.304000
2017-12-14 02  0.024265        0.041517  16065.750333
2017-12-14 03  0.029325        0.050666  15958.672667
2017-12-14 04  0.014558        0.040657  16082.817500
2017-12-14 05  0.031750        0.057571  16135.985500
2017-12-14 06  0.040616        0.060798  16214.115833
2017-12-14 07  0.021478        0.042819  16208.816667
2017-12-14 08  0.032020        0.052432  16626.591186
2017-12-14 09  0.022049        0.043830  16482.851591
2017-12-14 10  0.025644        0.057230  16442.608167
2017-12-14 11  0.010530        0.030502  16506.645333
2017-12-14 12  0.044814        0.061949  16639.402833
2017-12-14 13  0.012440        0.034257  16577.566333
2017-12-14 14  0.022767        0.041994  16389.246833
2017-12-14 15  0.029878        0.057725  16200.029167
2017-12-14 16  0.013018        0.042625  16364.761333
2017-12-14 17  0.028830        0.055072  16468.303000
2017-12-14 18  0

27227
19544
                Bitcoin  Bitcoin_median         Open
2018-04-03 08  0.046504        0.048628  7379.291000
2018-04-03 09  0.031409        0.032689  7363.867333
2018-04-03 10  0.046426        0.037049  7341.185167
2018-04-03 11  0.039005        0.035836  7320.024167
2018-04-03 12 -0.021520        0.001634  7341.528167
2018-04-03 13  0.019508        0.023100  7370.553333
2018-04-03 14  0.022925        0.023679  7326.757000
2018-04-03 15  0.040468        0.042255  7372.857833
2018-04-03 16  0.007848        0.014246  7397.795667
2018-04-03 17  0.018785        0.020117  7389.274500
2018-04-03 18  0.040503        0.037361  7447.432000
2018-04-03 19  0.031905        0.040541  7477.670500
2018-04-03 20  0.037352        0.045691  7449.060500
2018-04-03 21  0.034574        0.035011  7412.805833
2018-04-03 22 -0.006049        0.001147  7396.856167
2018-04-03 23  0.023896        0.023432  7415.387833
Iterration 25 of 65
34562
23728
                Bitcoin  Bitcoin_median         Open
20

27846
20537
                Bitcoin  Bitcoin_median         Open
2018-05-19 01  0.048061        0.056353  8202.069333
2018-05-19 02  0.043803        0.043428  8202.406000
2018-05-19 03  0.032934        0.047131  8227.218833
2018-05-19 04  0.077787        0.085839  8238.117667
2018-05-19 05  0.053826        0.041534  8230.143000
2018-05-19 06  0.037382        0.034883  8233.472667
2018-05-19 07  0.045192        0.034370  8202.284746
2018-05-19 08  0.057740        0.051336  8189.428000
2018-05-19 09  0.046520        0.047788  8211.836833
2018-05-19 10  0.057551        0.057311  8233.925667
2018-05-19 11  0.055746        0.044043  8251.552333
2018-05-19 12  0.035094        0.033946  8346.032333
2018-05-19 13  0.030901        0.032986  8325.539667
2018-05-19 14  0.045874        0.051977  8313.349833
2018-05-19 15  0.045006        0.042807  8301.755333
2018-05-19 16  0.057945        0.061224  8297.578475
2018-05-19 17  0.038335        0.034653  8294.249322
2018-05-19 18  0.012360        0.0

31494
24246
                Bitcoin  Bitcoin_median         Open
2018-06-19 01  0.028057        0.041520  6716.892542
2018-06-19 02  0.060959        0.045026  6710.740333
2018-06-19 03 -0.009741        0.005676  6685.136333
2018-06-19 04  0.064908        0.062753  6702.483390
2018-06-19 05  0.047217        0.037365  6702.170678
2018-06-19 06  0.053669        0.044778  6697.155833
2018-06-19 07  0.025368        0.020781  6689.229667
2018-06-19 08  0.003540        0.012844  6702.050000
2018-06-19 09  0.042090        0.043800  6705.717833
2018-06-19 10  0.023685        0.025666  6711.944500
2018-06-19 11  0.040168        0.042162  6734.480333
2018-06-19 12  0.052336        0.048997  6738.609500
2018-06-19 13  0.022011        0.024208  6780.443667
2018-06-19 14  0.040230        0.052205  6745.687333
2018-06-19 15  0.028167        0.040915  6745.768333
2018-06-19 16  0.042458        0.051301  6705.946500
2018-06-19 17  0.037587        0.043928  6714.383729
2018-06-19 18  0.062094        0.0

31299
23078
                Bitcoin  Bitcoin_median         Open
2018-07-23 01  0.063629        0.047770  7470.878621
2018-07-23 02  0.077544        0.070185  7605.024000
2018-07-23 03  0.043780        0.047815  7626.192833
2018-07-23 04  0.088870        0.089338  7657.812833
2018-07-23 05  0.062147        0.051117  7651.057833
2018-07-23 06  0.042865        0.031311  7667.666333
2018-07-23 07  0.039930        0.049687  7716.779167
2018-07-23 08  0.020156        0.023168  7717.842500
2018-07-23 09  0.040887        0.041624  7698.244167
2018-07-23 10  0.021963        0.024078  7696.202000
2018-07-23 11  0.038908        0.045612  7689.972167
2018-07-23 12  0.054205        0.070944  7679.216000
2018-07-23 13  0.026377        0.030430  7711.080333
2018-07-23 14  0.002548        0.004131  7703.962333
2018-07-23 15  0.022512        0.034250  7716.301509
2018-07-23 16  0.025536        0.031783  7723.356500
2018-07-23 17  0.021509        0.031774  7725.515667
2018-07-23 18  0.025894        0.0

25852
19617
                Bitcoin  Bitcoin_median         Open
2018-09-20 01  0.012164        0.016396  6406.450351
2018-09-20 02  0.007966        0.018701  6392.114231
2018-09-20 03 -0.010065        0.001348  6391.313269
2018-09-20 04 -0.015787        0.003001  6398.081500
2018-09-20 05  0.010041        0.019840  6391.634821
2018-09-20 06  0.000605        0.009777  6399.778182
2018-09-20 07  0.018500        0.033608  6403.422041
2018-09-20 08 -0.010514        0.003186  6393.782885
2018-09-20 09 -0.006410        0.000606  6390.821132
2018-09-20 10  0.015225        0.026595  6393.127288
2018-09-20 11  0.002180        0.023147  6419.140172
2018-09-20 12 -0.004443        0.015841  6431.185965
2018-09-20 13  0.026457        0.039067  6411.997333
2018-09-20 14  0.040893        0.057093  6417.856207
2018-09-20 15  0.050586        0.066861  6412.322931
2018-09-20 16  0.053392        0.061205  6416.941525
2018-09-20 17 -0.001659        0.026538  6412.316379
2018-09-20 18  0.004700        0.0

19053
12939
                Bitcoin  Bitcoin_median         Open
2018-12-30 01  0.016320        0.021411  3731.598814
2018-12-30 02  0.028359        0.027861  3737.729655
2018-12-30 03  0.031623        0.031563  3733.031200
2018-12-30 04  0.012606        0.012458  3743.063137
2018-12-30 05  0.058452        0.066936  3753.203273
2018-12-30 06  0.057257        0.037224  3755.017843
2018-12-30 07  0.039367        0.051029  3756.018511
2018-12-30 08  0.048148        0.043974  3756.648600
2018-12-30 09  0.050961        0.032556  3813.724423
2018-12-30 10  0.051224        0.065512  3829.143636
2018-12-30 11  0.037413        0.054854  3830.812857
2018-12-30 12  0.059914        0.052196  3805.376852
2018-12-30 13  0.010326        0.019215  3804.106735
2018-12-30 14  0.018642        0.030393  3802.066667
2018-12-30 15  0.024573        0.035530  3810.960588
2018-12-30 16  0.012643        0.024103  3802.030377
2018-12-30 17  0.038498        0.049963  3812.801429
2018-12-30 18  0.012391        0.0

# Evaluation and Testing (not important for now)

In [10]:
df2 = pd.read_csv(OUTPUT_DATASET_FILEPATH, index_col = 0)

In [11]:
df2

,Bitcoin,Bitcoin_median,Open
2017-09-13 01,0.005622,0.012747,4128.005439
2017-09-13 02,0.016173,0.017391,4066.801167
2017-09-13 03,-0.011676,0.000703,3999.225167
2017-09-13 04,0.004191,0.004836,3959.409667
2017-09-13 05,0.064276,0.058035,3995.046667
...,...,...,...
2019-01-22 18,0.021799,0.029650,3584.926897
2019-01-22 19,0.030228,0.035610,3585.858545
2019-01-22 20,0.014041,0.038498,3585.295893
2019-01-22 21,0.046292,0.066514,3579.910755


In [ ]:
DATASET_FILEPATH = "./Dataset_sentiment_100K_glove_filtered_2g.csv"
df2 = pd.read_csv(OUTPUT_DATASET_FILEPATH, index_col = 0)
fig = plt.figure(figsize=(18, 10))
plt.plot(range(0,100), -1*df2['Bitcoin'].values[:100])

In [ ]:
fig = plt.figure(figsize=(18, 10))
x = -df2['Bitcoin'][100:200]
x = x/x[0]
y = df2['Open'][100:200]
y = y/y[0]
plt.plot(range(0,100), x)
plt.plot(range(0,100), y)


In [ ]:
fig = plt.figure(figsize=(18, 10))
plt.plot(range(0,100), df.pct_change()['Open'].values[100:200])

In [ ]:
import random
i = random.randint(0,len(text_data.index))
print(text_data.loc[i]['text'])
print(text_data.loc[i]['sentiment'])

In [ ]:
#BTC price data of the next hour standarized
Y = pd.DataFrame(df['Open']).astype(float)
#Y = dataset_standarization(Y)

#Textual data ('Bitcoin' topic popularity) of the last hour standarized
X = pd.DataFrame(df['Bitcoin']).astype(float)
#X = dataset_standarization(X)
Y.index = X.index

X = sm.add_constant(X)

model = sm.OLS(Y, X, missing = 'drop').fit()

In [ ]:
model.summary()